#### Imports

In [1]:
import pandas as pd 
import re
from pathlib import Path
import create_dataset as cd
import feature_engineering as fe
import os 
import prep 
import numpy as np

In [2]:
data_tim = pd.read_csv('/Users/ryonamba/Documents/VU/VU-master_year_1/ML_quantified_self/ML4QS/datasets/raw_data_tim_copy.csv')
data_sien = pd.read_csv('/Users/ryonamba/Documents/VU/VU-master_year_1/ML_quantified_self/ML4QS/datasets/raw_data_sien_copy.csv')
data_tim2 = pd.read_csv('/Users/ryonamba/Documents/VU/VU-master_year_1/ML_quantified_self/ML4QS/datasets/raw_data_tim_2_copy.csv')
data_sien2 = pd.read_csv('/Users/ryonamba/Documents/VU/VU-master_year_1/ML_quantified_self/ML4QS/datasets/raw_data_sien_copy.csv')

data_tim = data_tim.drop(columns = ['Unnamed: 0'])
data_sien = data_sien.drop(columns = ['Unnamed: 0'])

In [3]:
proc_data = pd.read_csv('/Users/ryonamba/Documents/VU/VU-master_year_1/ML_quantified_self/ML4QS/datasets/processed_data.csv')
proc_data2 = pd.read_csv('/Users/ryonamba/Documents/VU/VU-master_year_1/ML_quantified_self/ML4QS/datasets/processed_data.csv')
proc_data.head()

,datetime,Time (s),X (m/s^2),Y (m/s^2),Z (m/s^2),X (rad/s),Y (rad/s),Z (rad/s),name_climber,grading,...,roll,pitch,Velocity_X,Velocity_Y,Velocity_Z,Displacement_X,Displacement_Y,Displacement_Z,distance,climb_id
0,2023-06-07 13:14:40.068719685-02:00,108.088635,0.157283,-0.442686,-1.072861,0.039662,-0.432147,-0.123366,0,4.3,...,-2.750253,0.134698,109.595352,76.050713,-76.762943,668.612258,-2206.086041,-2018.401478,3063.952105,1.1
1,2023-06-07 13:14:40.270581102-02:00,108.289898,-0.659116,0.043920,-0.302016,-0.675243,-0.943927,-0.006866,0,4.3,...,2.997183,-1.137157,109.462097,76.059137,-76.823308,690.639274,-2190.766094,-2033.852136,3068.060168,1.1
2,2023-06-07 13:14:40.472442519-02:00,108.491162,1.083198,-0.937163,0.811129,-0.707245,-0.285354,-0.007537,0,4.3,...,-0.857363,0.718231,109.680650,75.870898,-76.660439,712.717472,-2175.506899,-2049.291228,3072.521928,1.1
3,2023-06-07 13:14:40.674303937-02:00,108.692425,0.330704,0.027100,0.293211,-0.433973,-0.436695,-0.142976,0,4.3,...,0.092163,0.843308,109.746664,75.875975,-76.601045,734.801873,-2160.225063,-2064.698002,3077.252664,1.1
4,2023-06-07 13:14:40.876165354-02:00,108.893689,-0.816926,0.448517,-0.063428,-0.500497,-0.702771,0.051526,0,4.3,...,1.711283,-1.064510,109.582791,75.966622,-76.614192,756.860595,-2144.946450,-2080.127939,3082.298733,1.1


In [4]:
import json 

with open('heartrate/heart_rate-2023-06-07 copy.json', 'r') as f1: 
    hr_json1 = json.load(f1)
with open('heartrate/heart_rate-2023-06-14 copy.json', 'r') as f2: 
    hr_json2 = json.load(f2)

# put datetime & heart rate values in a list 
hr_data = [] 
for heart_entry in (hr_json1 + hr_json2):
    dt = heart_entry['dateTime']
    hr = heart_entry['value']['bpm']
    hr_data.append((dt, hr))

# prep fitbit datetimes 
fitbit_datetimes = [x[0] for x in hr_data]
fitbit_datetimes = pd.to_datetime(fitbit_datetimes)
# prep fitbit heartreates 
fitbit_heart_rates = [x[1] for x in hr_data]

# prep for comparing datetimes 
proc_data['datetime'] = pd.to_datetime(proc_data['datetime'])
proc_data['datetime'] = proc_data['datetime'].dt.tz_localize(None)
 

In [5]:

mapping_dict = {}

# iterates through each snippet 
for group_name, group_data in proc_data.groupby("climb_id"):
    # gets all fitbit datetimes within snippet time interval 
    start_time = group_data.iloc[0]['datetime']
    end_time = group_data.iloc[-1]['datetime']
    fitbit_snippet_datetimes = fitbit_datetimes[(fitbit_datetimes >= start_time) & (fitbit_datetimes <= end_time)]
    # match each fitbit datetime to nearest datetime in the processed dataframe 
    for fitbit_dt in fitbit_snippet_datetimes:
        # datetime matching 
        closest_index = abs(proc_data['datetime'] - fitbit_dt).idxmin()
        data_closest_dt = proc_data.loc[closest_index, 'datetime']
        # get corresponding heart rate values & save mapping 
        hr_idx = fitbit_datetimes.get_loc(fitbit_dt)
        mapping_dict[data_closest_dt] = fitbit_heart_rates[hr_idx] 


In [6]:
# Add new column by using HR to datetime mapping 
proc_data['heart-rate'] = proc_data['datetime'].map(mapping_dict)
proc_data
proc_data['heart-rate'].nunique()


73

In [7]:
# Perform linear interpolation within each climb ID 
proc_data['heart-rate'] = proc_data.groupby('climb_id', group_keys=False)['heart-rate'].apply(lambda x: x.interpolate(limit_direction='both'))
proc_data


,datetime,Time (s),X (m/s^2),Y (m/s^2),Z (m/s^2),X (rad/s),Y (rad/s),Z (rad/s),name_climber,grading,...,pitch,Velocity_X,Velocity_Y,Velocity_Z,Displacement_X,Displacement_Y,Displacement_Z,distance,climb_id,heart-rate
0,2023-06-07 13:14:40.068719685,108.088635,0.157283,-0.442686,-1.072861,0.039662,-0.432147,-0.123366,0,4.3,...,0.134698,109.595352,76.050713,-76.762943,668.612258,-2206.086041,-2018.401478,3063.952105,1.1,91.0
1,2023-06-07 13:14:40.270581102,108.289898,-0.659116,0.043920,-0.302016,-0.675243,-0.943927,-0.006866,0,4.3,...,-1.137157,109.462097,76.059137,-76.823308,690.639274,-2190.766094,-2033.852136,3068.060168,1.1,91.0
2,2023-06-07 13:14:40.472442519,108.491162,1.083198,-0.937163,0.811129,-0.707245,-0.285354,-0.007537,0,4.3,...,0.718231,109.680650,75.870898,-76.660439,712.717472,-2175.506899,-2049.291228,3072.521928,1.1,91.0
3,2023-06-07 13:14:40.674303937,108.692425,0.330704,0.027100,0.293211,-0.433973,-0.436695,-0.142976,0,4.3,...,0.843308,109.746664,75.875975,-76.601045,734.801873,-2160.225063,-2064.698002,3077.252664,1.1,91.0
4,2023-06-07 13:14:40.876165354,108.893689,-0.816926,0.448517,-0.063428,-0.500497,-0.702771,0.051526,0,4.3,...,-1.064510,109.582791,75.966622,-76.614192,756.860595,-2144.946450,-2080.127939,3082.298733,1.1,91.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9195,2023-06-14 17:50:12.884499151,92.831709,-0.030619,-0.072741,0.378106,-0.001807,0.039379,-0.054433,0,6.0,...,-0.079354,59.438698,89.631169,-154.750685,3596.124410,-5710.731598,3054.774172,7407.847960,47.2,92.0
9196,2023-06-14 17:50:13.085979626,93.032993,-0.026855,0.064846,0.292096,-0.057355,0.051656,-0.023956,0,6.0,...,-0.089512,59.433476,89.644497,-154.692367,3608.098472,-5692.705219,3023.646542,7386.995716,47.2,92.0
9197,2023-06-14 17:50:13.287460101,93.234276,-0.014772,0.014911,0.115421,-0.025292,-0.013718,0.001490,0,6.0,...,-0.126258,59.430322,89.647227,-154.668666,3620.049851,-5674.643504,2992.505204,7366.235620,47.2,92.0
9198,2023-06-14 17:50:13.488940577,93.435560,0.038016,-0.093744,0.075088,0.005001,-0.026046,-0.003751,0,6.0,...,0.306535,59.438266,89.628798,-154.654313,3632.031628,-5656.630571,2961.390484,7345.676059,47.2,92.0


In [13]:
temp = prep.add_heart_rate(proc_data2)
proc_data.equals(temp)

{Timestamp('2023-06-07 13:14:43.904086614'): 91, Timestamp('2023-06-07 13:14:48.950622047'): 92, Timestamp('2023-06-07 13:14:53.997157480'): 91, Timestamp('2023-06-07 13:14:59.043692913'): 90, Timestamp('2023-06-07 13:14:08.982061417'): 88, Timestamp('2023-06-07 13:14:14.028596850'): 87, Timestamp('2023-06-07 13:14:19.075132283'): 88, Timestamp('2023-06-07 13:14:23.919806299'): 89, Timestamp('2023-06-07 13:30:03.924619771'): 91, Timestamp('2023-06-07 13:30:08.965636882'): 91, Timestamp('2023-06-07 13:30:14.006653992'): 90, Timestamp('2023-06-07 13:30:19.047671102'): 89, Timestamp('2023-06-07 13:28:48.914285171'): 104, Timestamp('2023-06-07 13:28:53.955302281'): 100, Timestamp('2023-06-07 13:28:58.996319391'): 103, Timestamp('2023-06-07 13:29:04.037336501'): 101, Timestamp('2023-06-07 13:39:40.914571056'): 91, Timestamp('2023-06-07 13:39:50.992146023'): 89, Timestamp('2023-06-07 13:39:56.030933507'): 91, Timestamp('2023-06-07 13:39:09.069434159'): 97, Timestamp('2023-06-07 13:39:13.9066

True

In [9]:
data = cd.combine_start_dfs(data_sien, data_sien2, data_tim, data_tim2)
# change name climber to numerical value
data['name_climber'] = data['name_climber'].map({'sien':0,'tim':1})
# drop weird column (SIEN's FAULT)
data = data.drop(columns = ['Time (s).1'])
# drop climber rating var 
data = cd.drop_climber_rating(data)
# add datetime var 
data = cd.add_datetime(data)
# add heart-rate var



# cut falls 
# data = cd.cut_fall(data)

# data = fe.features(data)
# data.grading = data.grading.apply(fe.french_grade_to_num)

# data = create_snippets(data, 20, 5)
# data = change_climb_ID(data)

data

,entry_num,datetime,Time (s),X (m/s^2),Y (m/s^2),Z (m/s^2),X (rad/s),Y (rad/s),Z (rad/s),name_climber,grading,num_attempt,fall_top,outdoor,Unnamed: 0
0,15,2023-06-07 16:36:31.299000-02:00,0.202257,0.616807,0.545096,-0.119170,-0.738735,-0.693665,-0.053444,0,4c,1,0,0.0,NaN
1,15,2023-06-07 16:36:31.399774729-02:00,0.403552,0.427085,-0.449999,0.075464,-0.314339,-0.378076,0.015612,0,4c,1,0,0.0,NaN
2,15,2023-06-07 16:36:31.500549459-02:00,0.604846,0.570240,0.152899,-0.213106,-0.273812,-0.442562,-0.130227,0,4c,1,0,0.0,NaN
3,15,2023-06-07 16:36:31.601324189-02:00,0.806141,0.295633,-0.453686,0.244799,-0.260747,-0.114135,-0.019194,0,4c,1,0,0.0,NaN
4,15,2023-06-07 16:36:31.702098919-02:00,1.007435,0.206272,-0.927811,-0.512616,-0.160211,-0.226173,-0.211577,0,4c,1,0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5186,40,2023-06-14 17:02:20.736479365-02:00,62.783481,0.138590,-1.738872,-0.396787,-0.371624,-0.194527,-0.130351,1,5b,1,0,NaN,NaN
5187,40,2023-06-14 17:02:20.938609523-02:00,62.984758,-0.756452,-0.380505,0.312611,0.064059,0.111573,-0.053014,1,5b,1,0,NaN,NaN
5188,40,2023-06-14 17:02:21.140739682-02:00,63.186036,-0.321295,-0.189666,-0.070279,-0.062600,-0.059749,-0.085358,1,5b,1,0,NaN,NaN
5189,40,2023-06-14 17:02:21.342869841-02:00,63.387314,-0.421361,-0.344314,0.504001,0.049513,-0.251242,0.149490,1,5b,1,0,NaN,NaN


#### Heart rate

- time jumps are inconsistent (sometimes 5, sometimes 10)
- extract & map heart rate values within a datetime range to their group -> dict? 
    - iterate through df groups
        - iterate through json file 
        
- fit values to entries in each group (will have some nans)
- interpolate missing values 

In [10]:
import json 

with open('heartrate/heart_rate-2023-06-07 copy.json', 'r') as f1: 
    hr_json1 = json.load(f1)
with open('heartrate/heart_rate-2023-06-14 copy.json', 'r') as f2: 
    hr_json2 = json.load(f2)


# convert entries to datetime obj & put values in list 
hr_data1 = [] 
for heart_entry in hr_json1:
    dt = heart_entry['dateTime']
    hr = heart_entry['value']['bpm']
    hr_data1.append((dt, hr))

hr_data2 = []
for heart_entry in hr_json2:
    dt = heart_entry['dateTime']
    hr = heart_entry['value']['bpm']
    hr_data2.append((dt, hr))
 

In [11]:
import json 

with open('heartrate/heart_rate-2023-06-07 copy.json', 'r') as f1: 
    hr_json1 = json.load(f1)
with open('heartrate/heart_rate-2023-06-14 copy.json', 'r') as f2: 
    hr_json2 = json.load(f2)


# convert entries to datetime obj & put values in list 
hr_data1 = [] 
for heart_entry in hr_json1:
    dt = heart_entry['dateTime']
    hr = heart_entry['value']['bpm']
    hr_data1.append((dt, hr))

hr_data2 = []
for heart_entry in hr_json2:
    dt = heart_entry['dateTime']
    hr = heart_entry['value']['bpm']
    hr_data2.append((dt, hr))
 

In [12]:
fit_datetimes = [x[0] for x in hr_data1]
fit_hr = [x[1] for x in hr_data1]

# convert fitbit times to datetime obj
fit_datetimes = pd.to_datetime(fit_datetimes)

# NOTE: OK! but need to do this for only the datetimes within a certain interval, cuz otherwise it's gonna match each row in the df
# NOTE: maybe try not to merge dfs, but init a df and then do the masks
# NOTE: adapt this to extract heartrate shit then  
# NOTE: it seems like 1 climb had no heart rate measurements (oh wait it's cuz I'm ony using part of the data)
# NOTE: make sure to do interpolation per grouping 
# NOTE: datetime in data is every 100ms now instead of 200ms 

# inits null heartrate variable 
# if not 'heart-rate' in data:
#     data.insert(2, 'heart-rate', float("nan"))

# group by climbID & get datetime intervals 
for group_name, group_data in data.groupby("entry_num"):
    print(group_name)
    
    # gets all fitbit datetimes within the given interval 
    start_time = group_data.iloc[0]['datetime']
    end_time = group_data.iloc[-1]['datetime']
    filtered_dt = fit_datetimes[(fit_datetimes >= start_time) & (fit_datetimes <= end_time)]

    print(start_time, end_time)
    group_data[['datetime', 'Time (s)']].to_csv('temp.csv')
    

    # gets heart-rate values for given datetime interval 
    filtered_idxs = np.where(np.isin(fit_datetimes, filtered_dt))[0]
    filtered_hr = fit_hr[filtered_idxs[0]:filtered_idxs[-1] + 1]

    # TODO: calculation here seems to mess up -> when moving to new minute


    # map & retrieve closests df datetimes to fitbit datetimes per group 
    # # print(filtered_hr)
    # # make df out of fitbit dt and hr 
    # filtered_df = pd.DataFrame(list(zip(filtered_dt, filtered_hr)), columns=['datetime', 'heart-rate'])
    # print(filtered_df)
    # merged_df = pd.merge_asof(group_data, filtered_df, left_on='datetime', right_on='datetime', direction='nearest', suffixes=('', '_drop'))
    # merged_df = merged_df.drop(columns=merged_df.filter(regex='_drop$').columns)
    # print(merged_df)
    # merged_df.to_csv('temp.csv')


    # -- old -- 
    data = data.sort_values('datetime')

    closest_datetimes = filtered_dt.map(lambda x: data['datetime'].iloc[(data['datetime'] - x).abs().idxmin()])


    comp_dt = [(f'fit: {i}', f'cloests {j}') for i, j in zip(filtered_dt, closest_datetimes)]
    print(comp_dt)


    # # mask based on the matching datetime values
    # mask = data['datetime'].isin(closest_datetimes)
    # # adding heart-rate values to df 
    # filtered_dt = data[mask]
    # data.loc[mask, 'C'] = filtered_hr

    break 

# TODO: I don't think this is working quite right ... 
temp = data[data['entry_num'] == 1]
temp

    #     # fills in datetime sequences
    # for entry_id in df_copy["entry_num"].unique():
    #     mask = df_copy["entry_num"] == entry_id
    #     df_copy.loc[mask, "datetime"] = datetime_sequence_dict[entry_id]

    

    # Filter the dataframe based on the datetime interval
    # filtered_dt = data[(df['timestamp'] >= start_datetime) & (df['timestamp'] <= end_datetime)]





# Find closest respective dates
# closest_dates = fit_datetimes.map(lambda x: data['datetime'].iloc[(data['datetime'] - x).abs().idxmin()])
# print(closest_dates)


# # Create a DataFrame with the closest respective dates
# df_new = pd.DataFrame({'closest_datetime': closest_dates, 'new_datetime': fit_datetimes})

# # Merge the original DataFrame with the closest respective dates
# merged_df = pd.merge(data, df_new, left_on='datetime', right_on='closest_datetime', how='outer')

# merged_df




1


TypeError: Invalid comparison between dtype=datetime64[ns] and Timestamp

In [ ]:

new_datetimes = [x[0] for x in hr_data1]
hr = [x[1] for x in hr_data1]

# iterate through climb IDs
for group_name, group_data in data.groupby("entry_num"):
    print(group_name)
    print(group_data.iloc[0]['datetime'])
    print(group_data.iloc[-1]['datetime'])

    new_datetimes = pd.to_datetime(new_datetimes)

    # closest_dates = new_datetimes.map(lambda x: group_data['datetime'].iloc[(group_data['datetime'] - x).abs().idxmin()])


    # iterate through heart data & match entries by closest datetime 
    # for heart_entry in heart_data1:
    #     print(heart_entry)
    

    # print(type(heart_data1[0]['dateTime']))
    # for entry in heart_data1:
        # if entry['dateTime'] in group_data['datetime']:
        #     print('asdfasdf')
    break   




In [ ]:
import pandas as pd

# Original DataFrame with datetime entries
df_original = pd.DataFrame({
    'datetime_original': pd.to_datetime(['2023-06-19 12:00:00', '2023-06-19 13:30:00', '2023-06-19 14:45:00']),
    'value_original': [1, 2, 3]
})

# New datetime entries
new_datetimes = pd.to_datetime(['2023-06-19 12:15:00', '2023-06-19 13:40:00'])
print(new_datetimes)

# Find closest respective dates
closest_dates = new_datetimes.map(lambda x: df_original['datetime_original'].iloc[(df_original['datetime_original'] - x).abs().idxmin()])

# Create a DataFrame with the closest respective dates
df_new = pd.DataFrame({'closest_datetime': closest_dates, 'new_datetime': new_datetimes})

# Merge the original DataFrame with the closest respective dates
merged_df = pd.merge(df_original, df_new, left_on='datetime_original', right_on='closest_datetime', how='outer')

merged_df



In [ ]:
data_tim = pd.read_csv('/Users/ryonamba/Documents/VU/VU-master_year_1/ML_quantified_self/ML4QS/datasets/raw_data_tim_copy.csv')
data_sien = pd.read_csv('/Users/ryonamba/Documents/VU/VU-master_year_1/ML_quantified_self/ML4QS/datasets/raw_data_sien_copy.csv')
data_tim2 = pd.read_csv('/Users/ryonamba/Documents/VU/VU-master_year_1/ML_quantified_self/ML4QS/datasets/raw_data_tim_2_copy.csv')
data_sien2 = pd.read_csv('/Users/ryonamba/Documents/VU/VU-master_year_1/ML_quantified_self/ML4QS/datasets/raw_data_sien_copy.csv')


data_tim = data_tim.drop(columns = ['Unnamed: 0'])
data_sien = data_sien.drop(columns = ['Unnamed: 0'])




df = cd.combine_start_dfs(data_sien, data_sien2, data_tim, data_tim2)
df['name_climber'] = df['name_climber'].map({'sien':0,'tim':1})
df
df = df.drop(columns = ['Time (s).1'])
df = cd.drop_climber_rating(df)
df = cd.add_datetime(df)
df = cd.cut_fall(df)


df.grading = df.grading.apply(fe.french_grade_to_num)


df


In [ ]:
# code from DMT
# datatypes - NOTE: subject to change (only accelerometer & gyrometer rn)
types = ['id','datetime', 'numeric', 'numeric', 'numeric', 'numeric', 'numeric', 'numeric', 'categorical', 'ordinal', ] 

# nb of values 
n_values = [data[var].count() for var in data]

# nb of unique values
n_unique = [data[var].nunique() for var in data]


# TODO: do once we have the final variables (eg. for grade)
## mean + std 
# mean_std = []

# for var in data: 
#     if var not in ['datetime', 'entry_num']: # TODO 
#         mean = round(data[var].mean(), 2)
#         std = round(data[var].std(), 2) 
#         res = f'{mean} ({std})'
#         mean_std.append(res)
# mean_std.insert(0,'Na')
# mean_std.insert(0,'Na')

# # range of values
# val_range = []
# for var in data:
#     if var != 'id':
#         r = (data[var].min(), data[var].max())
#         if var != 'time':
#             r = (round(r[0]), round(r[1]))
#         else:
#             r = (data['time'].dt.date.min(), data['time'].dt.date.max())
#         val_range.append(r)
#         r = data['time'].dt.date
# val_range.insert(0, 'Na')

# # date range 
# time_range = []
# for var in data: 
#     # if var != 'id':
#     grouped = data.groupby(var)['time'].min()
#     r = (grouped.dt.date.min(), grouped.dt.date.max())
#     time_range.append(r)


# # missing values before time aggregation TODO: do for heartrate only ig
# missing_before_agg = data.isna().sum().tolist()

# # missing values after time aggregation -> fill this in manually... 
# # automatic 14 for each except call & sms 
# # + 13 for activity 
# # + 1 for 'circumplex.arousal','circumplex.valence', 'mood' & -2 cuz there were like 2 days of measurements in the first 14 days? 
# # temp_data = fn.group_data(data, count=False)
# # missing_after_agg = temp_data.isna().sum().tolist()
# # missing_after_agg.insert(0, 0)


# missing_after_agg = [0 for i in range(len(data.columns))]
# for i, var in enumerate(data.columns):
#     if var == 'sms' or var == 'call' or var == 'id' or var == 'time': 
#         continue
#     if var == 'circumplex.arousal' or var == 'circumplex.valence' or var == 'mood':
#         missing_after_agg[i] += 1
#     missing_after_agg[i] += 14
#     if var == 'activity':
#         missing_after_agg[i] += 13



data_table = pd.DataFrame({'Variables': data.columns, 
                    # 'Data types': types, 
                    'number values': n_values, 
                    'number unique values': n_unique, 
                    # 'Mean (std)': mean_std,
                    # 'Value range': val_range,
                    # 'Date range': time_range, 
                    # 'Missing values (before time aggregation)': missing_before_agg,
                    # 'Missing values (after time aggregation)': missing_after_agg
                    })
data_table